In [1]:
import pandas as pd
train_csv = 'train.csv'
train_df = pd.read_csv(train_csv)
# To Do: sort the df based on size of comments (no. of words in comment)

#### Training Data Preperation
* read the labels and convert into one-class labels
* we will focus on 2 class problem: toxic and non toxic comments
* we will label all different types of toxic comments into same category of toxic label:
    * 0 for toxic comment
    * 1 for non-toxic comments
* later we can explore how to make it multiclass classifier

In [2]:
# each toxic class is labelled as 1
toxic_row_sums = train_df.iloc[:,2:].sum(axis=1)
# if sum of toxic class is 0 then it is a clean comment
train_df['clean'] = (toxic_row_sums==0)
# Input Data
train_texts = train_df['comment_text']
# Output Label
train_labels = train_df['clean']

### Pre-processing : Tokenization
Now we have training data in two separate dataframe columns (arrays/list): an ordered array consisting of comments (input for the network) and another array consisting of class lables in same order (output of the network).

We have to transform this data into network input format and output format. This step is called pre-processing.  
Steps of pre-processing:

1. Tokenize the text into words
2. Assign each word a dimension


To accompolish step 1 and 2 we will use inbuilt __Tokenizer__ class

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
# set size of vocabulary
# To Do: try different size 
max_vocab_size = 10000
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(train_texts)
sequences = tokenizer.texts_to_sequences(train_texts)
print(sequences[0])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


[688, 75, 1, 126, 130, 177, 29, 672, 4511, 1116, 86, 331, 51, 2278, 50, 6864, 15, 60, 2756, 148, 7, 2937, 34, 117, 1221, 2825, 4, 45, 59, 244, 1, 365, 31, 1, 38, 27, 143, 73, 3462, 89, 3085, 4583, 2273, 985]
Found 210337 unique tokens.


### Batching and Padding for Embedding
Now once we have the tokens and each token(word) has a dimension assigned to it, we will do following steps to create word embeddings  

3. use this dimension assignments to define embedding for individual word
4. use word embedding to create word vector for a comment


We will use a specific type of network layer for this, which is called __Embedding Layer__. The above generated tokens (sequence of number) will go as input to Embedding layer, which will output word embeddings as output to next layer.  

Input and Output of Neural Network are done is batches. A batch is a group of input data which are fed together to the network. As the network can process individual data element in parallel, the training will be faster.

In case of Embedding Layer, Inpupt and Output in a batch can be seen as follows:  

   **Input**: 2D tensor of integers, of shape (# seq. samples in particular batch, sequence_length), where each entry is a sequence of integers (output of above code).  
   **Output**: 3D floating-point tensor of shape (# seq. samples in particula patch, sequence_length, embedding_dimensionality).  

Sequence length can be variable per batch. But in a single batch sequence length will be same for all sequences.  

So from data we have to create batches of sequence of similar length and to do that we have to pad or truncate each sequence to have same sequence length. And we can use each batch as a training input for embedding layer.  

For sample case: we take 10k sequence from 160k for training in a single batch. And take max sequence length of 20 words.


In [4]:
from tensorflow.keras import preprocessing
training_sequences = sequences[:10000]
training_labels = train_labels[:10000]
seq_max_len = 20
# training padded sequences
train_seq_pad = preprocessing.sequence.pad_sequences(sequences=training_sequences, maxlen=seq_max_len)

# testing padded sequences
testing_sequences = sequences[10000:11000]
testing_labels = train_labels[10000:11000]
test_seq_pad = preprocessing.sequence.pad_sequences(sequences=testing_sequences, maxlen=seq_max_len)

# To Do: try more training data, try different sequence max length

### Model 3: Embedding => Bidirectional RNN => Output
In this model 3 we will extend the Model 2 by wrapping the RNN layer with a Bidirectional wrapper.

#### Define the model 3
Model 3 is made of 4 layers:
    - Layer 0 is input layer
    - Layer 1 is Embedding layer
    - Layer 2 is Bidirectional RNN layer (return last output)
    - Layer 3 is Dense Layer (output/classification layer)  

**Bidirectional Layer**  
Bidirectional layer of two hidden layers with opposite direction for input, it processes the sequence in both order backward and forward and generates a combined output.  
Ref for more details: 
 1. https://en.wikipedia.org/wiki/Bidirectional_recurrent_neural_networks
 2. https://d2l.ai/chapter_recurrent-neural-networks/bi-rnn.html


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
from tensorflow.keras.layers import Bidirectional

# model configurations
vocab_size = 10000
seq_max_len = 20 # this can be removed as it is not required for next layer which is RNN
embedding_dim = 16

# model definition
model_3 = Sequential()
model_3.add(Embedding(vocab_size, embedding_dim, input_length=seq_max_len))
# [1] This will create two copies of the hidden layer, 
# one fit in the input sequences as-is and one on a reversed copy of the input sequence. 
# By default, the output values from these LSTMs will be concatenated.
model_3.add(Bidirectional(SimpleRNN(32)))
model_3.add(Dense(1, activation='sigmoid'))
model_3.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [6]:
model_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            160000    
                                                                 
 bidirectional (Bidirection  (None, 64)                3136      
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 163201 (637.50 KB)
Trainable params: 163201 (637.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Train model 3

In [7]:
import numpy as np
history_3 = model_3.fit(train_seq_pad, np.asarray(training_labels), epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


250/250 [==============================] - 9s 20ms/step - loss: 0.2875 - acc: 0.9031 - val_loss: 0.2114 - val_acc: 0.9285
Epoch 2/10
250/250 [==============================] - 4s 15ms/step - loss: 0.1875 - acc: 0.9356 - val_loss: 0.2505 - val_acc: 0.9135
Epoch 3/10
250/250 [==============================] - 4s 16ms/step - loss: 0.1398 - acc: 0.9538 - val_loss: 0.2004 - val_acc: 0.9365
Epoch 4/10
250/250 [==============================] - 4s 16ms/step - loss: 0.1067 - acc: 0.9630 - val_loss: 0.2330 - val_acc: 0.9225
Epoch 5/10
250/250 [==============================] - 4s 17ms/step - loss: 0.0786 - acc: 0.9736 - val_loss: 0.2343 - val_acc: 0.9275
Epoch 6/10
250/250 [==============================] - 5s 22ms/step - loss: 0.0524 - acc: 0.9841 - val_loss: 0.2792 - val_acc: 0.9190
Epoch 7/10
250/250 [==============================] - 5s 21ms/step - loss: 0.0350 - acc: 0.9894 - val_loss: 0.2898 - val_acc: 0.9235
Epoch 8/10
250/250 [==============================] - 6s 22ms/step 

#### Testing model 3

In [8]:
print(model_3.metrics_names)
model_3.evaluate(x=test_seq_pad, y=np.asarray(testing_labels))

['loss', 'acc']
32/32 [==============================] - 0s 6ms/step - loss: 0.4900 - acc: 0.8680


[0.4900113642215729, 0.8679999709129333]

In [9]:
model_3.save('toxic.h5')

c:\Users\Kowshik Rayani\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
from tensorflow.keras.models import load_model
model1 = load_model('toxic.h5')


In [11]:
#test_texts = ["D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"]
test_texts = ["how do u motherf**k"]
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_seq_pad = preprocessing.sequence.pad_sequences(sequences=test_sequences, maxlen=seq_max_len)

#predictions = model1.predict(test_seq_pad)
print(test_texts)

['how do u motherf**k']


In [12]:
predictions = model1.predict(test_seq_pad, batch_size=1)


1/1 [==============================] - 1s 761ms/step


In [13]:
threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)

In [14]:
print(binary_predictions)

[[1]]


In [15]:
num_non_toxic = np.sum(binary_predictions == 1)
num_toxic = np.sum(binary_predictions == 0)
total_comments = len(binary_predictions)

non_toxic_percentage = (num_non_toxic / total_comments) * 100
toxic_percentage = (num_toxic / total_comments) * 100

print(f"Non-Toxic Comments: {non_toxic_percentage}%")
print(f"Toxic Comments: {toxic_percentage}%")


Non-Toxic Comments: 100.0%
Toxic Comments: 0.0%
